In [9]:
import sqlite3
import pandas as pd
import os
import sys

import sqlite3
import pandas as pd
import os
import sys
from typing import List, Tuple, Optional, Any # For type hinting


class DatabaseManager: # Renamed class for slightly more generic feel
    """
    Manages SQLite database operations using generic methods and OOP.

    Handles connection lifecycle, executing SQL commands (DDL/DML),
    importing data from Excel, and executing SELECT queries.
    Designed for use as a context manager.

    Attributes:
        db_path (str): Path to the SQLite database file.
        excel_path (Optional[str]): Path to the associated Excel data source.
        conn (Optional[sqlite3.Connection]): Active SQLite connection.
        cursor (Optional[sqlite3.Cursor]): Cursor for executing SQL.
    """

    def __init__(self, db_path: str, excel_path: Optional[str] = None):
        """
        Initializes the DatabaseManager.

        Args:
            db_path (str): The path for the SQLite database file.
            excel_path (Optional[str]): Path to the Excel file (if needed).
        """
        self.db_path = db_path
        self.excel_path = excel_path
        self.conn = None
        self.cursor = None

    def connect(self) -> None:
        """Establishes the database connection and initializes the cursor."""
        if self.conn: # Avoid reconnecting if already connected
            print("INFO: Connection already established.")
            return
        try:
            print(f"INFO: Connecting to database: '{self.db_path}'...")
            self.conn = sqlite3.connect(self.db_path)
            self.cursor = self.conn.cursor()
            print("SUCCESS: Database connection established.")
        except sqlite3.Error as e:
            print(f"ERROR: Failed to connect to database '{self.db_path}'. Error: {e}", file=sys.stderr)
            self.conn = None
            self.cursor = None
            raise ConnectionError(f"Database connection failed: {e}") from e

    def close(self) -> None:
        """Commits changes and closes the database connection if open."""
        if self.conn:
            try:
                print("INFO: Committing final changes and closing connection...")
                self.conn.commit() # Ensure last changes are saved
                self.conn.close()
                print(f"SUCCESS: Database connection to '{self.db_path}' closed.")
            except sqlite3.Error as e:
                print(f"WARN: Error during closing sequence: {e}", file=sys.stderr)
            finally:
                self.conn = None
                self.cursor = None
        else:
            print("INFO: No active connection to close.")

    def __enter__(self):
        """Enters the runtime context, establishing the connection."""
        self.connect() # Ensure connection is active
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Exits the runtime context, ensuring connection closure."""
        print("INFO: Exiting context manager...")
        self.close()
        if exc_type:
             print(f"ERROR: Exception occurred within context: {exc_type.__name__}({exc_val})", file=sys.stderr)
        return False # Propagate exceptions

    def execute_sql(self, sql_command: str, params: Optional[Tuple[Any, ...]] = None) -> None:
        """
        Executes a given SQL command (e.g., CREATE, INSERT, UPDATE, DELETE).
        Does NOT automatically commit changes.

        Args:
            sql_command (str): The SQL statement to execute.
            params (Optional[Tuple[Any, ...]]): Optional parameters for the query.

        Raises:
            ConnectionError: If the database connection/cursor is not active.
            sqlite3.Error: If the SQL execution fails.
        """
        if not self.cursor:
            raise ConnectionError("Database cursor is unavailable. Cannot execute SQL.")
        try:
            print("INFO: Executing SQL command...")
            if params:
                self.cursor.execute(sql_command, params)
            else:
                self.cursor.execute(sql_command)
            # print(f"DEBUG: SQL executed: {sql_command[:100]}...") # Optional debug
            print("SUCCESS: SQL command executed.")
        except sqlite3.Error as e:
            print(f"ERROR: Failed to execute SQL command. Error: {e}", file=sys.stderr)
            print(f"SQL: {sql_command}", file=sys.stderr)
            if params: 
                print(f"Params: {params}", file=sys.stderr)
            raise # Re-raise the error after logging

    def commit(self) -> None:
        """Commits the current database transaction."""
        if not self.conn:
            raise ConnectionError("Database connection is unavailable. Cannot commit.")
        try:
            print("INFO: Committing transaction...")
            self.conn.commit()
            print("SUCCESS: Transaction committed.")
        except sqlite3.Error as e:
            print(f"ERROR: Failed to commit transaction. Error: {e}", file=sys.stderr)
            raise

    def import_data_from_excel(self, table_name: str, expected_columns: List[str]) -> bool:
        """
        Imports data from the instance's Excel file path into the specified table.
        Handles data validation, formatting, and commits the transaction on success.

        Args:
            table_name (str): The name of the target table.
            expected_columns (List[str]): List of column names expected in the Excel file,
                                          in the order they appear in the target table.

        Returns:
            bool: True if import was successful or file was empty/not found,
                  False if a critical error occurred.

        Raises:
            ConnectionError: If the database connection/cursor is not active.
            ValueError: If the Excel path was not provided during initialization.
        """
        print(f"\n--- Importing Data from Excel '{self.excel_path}' into '{table_name}' ---")
        if not self.cursor:
            raise ConnectionError("Database cursor is unavailable. Cannot import data.")
        if not self.excel_path:
             raise ValueError("Excel file path was not provided during initialization.")
        if not os.path.exists(self.excel_path):
            print(f"WARN: Excel file '{self.excel_path}' not found. No data imported.")
            return True

        try:
            df = pd.read_excel(self.excel_path, engine='openpyxl')
            print(f"INFO: Read {len(df)} rows from '{self.excel_path}'.")
            if df.empty:
                 print("INFO: Excel file is empty. No data to import.")
                 return True

            # --- Data Preprocessing & Validation ---
            missing_cols = [col for col in expected_columns if col not in df.columns]
            if missing_cols:
                print(f"ERROR: Excel file is missing required columns: {missing_cols}. Import aborted.", file=sys.stderr)
                return False

            # Ensure HireDate is formatted correctly (YYYY-MM-DD text) - Specific to this dataset
            if 'HireDate' in expected_columns and 'HireDate' in df.columns:
                original_dtype = df['HireDate'].dtype
                try:
                    df['HireDate'] = pd.to_datetime(df['HireDate'], errors='coerce').dt.strftime('%Y-%m-%d')
                    print("INFO: 'HireDate' column processed for YYYY-MM-DD format.")
                except Exception as date_err:
                     print(f"WARN: Issue processing 'HireDate' (type: {original_dtype}): {date_err}", file=sys.stderr)

            # Select/order columns, handle NaN/NaT -> None
            df_ordered = df[expected_columns].where(pd.notnull(df), None)
            data_to_insert = [tuple(row) for row in df_ordered.to_numpy()]

            # --- Database Insertion ---
            # Prepare placeholders (?, ?, ...) based on number of columns
            placeholders = ", ".join(["?"] * len(expected_columns))
            cols_string = ", ".join(expected_columns)
            # Use INSERT OR REPLACE strategy as in the original code
            insert_sql = f"INSERT OR REPLACE INTO {table_name} ({cols_string}) VALUES ({placeholders});"

            print("INFO: Executing bulk insert...")
            self.cursor.executemany(insert_sql, data_to_insert)
            self.commit() # Commit the import transaction
            print(f"SUCCESS: Data imported into '{table_name}'. {len(data_to_insert)} rows affected.")
            return True

        except FileNotFoundError: # Should be caught by os.path.exists, but good practice
             print(f"WARN: Excel file '{self.excel_path}' suddenly not found during read. No data imported.")
             return True
        except pd.errors.EmptyDataError:
            print("INFO: Excel file is empty. No data imported.")
            return True
        except Exception as e:
            print(f"FAILURE: An error occurred during data import: {e}", file=sys.stderr)
            try:
                if self.conn: 
                    self.conn.rollback() # Roll back on error
                print("INFO: Database transaction rolled back.")
            except sqlite3.Error as rb_err:
                print(f"WARN: Error during rollback: {rb_err}", file=sys.stderr)
            return False

    def execute_select_query(self, sql_query: str, params: Optional[Tuple[Any, ...]] = None) -> Optional[List[Tuple]]:
        """
        Executes a SELECT query and fetches all results.

        Args:
            sql_query (str): The SELECT SQL query string.
            params (Optional[Tuple[Any, ...]]): Optional parameters for the query.

        Returns:
            Optional[List[Tuple]]: A list of tuples representing the rows fetched,
                                   or None if an error occurs during execution.

        Raises:
            ConnectionError: If the database connection/cursor is not active.
        """
        if not self.cursor:
            raise ConnectionError("Database cursor is unavailable. Cannot execute query.")
        try:
            print("INFO: Executing SELECT query...")
            if params:
                self.cursor.execute(sql_query, params)
            else:
                self.cursor.execute(sql_query)
            results = self.cursor.fetchall()
            print(f"SUCCESS: SELECT query executed. Found {len(results)} records.")
            return results
        except sqlite3.Error as e:
            print(f"ERROR: Failed to execute SELECT query. Error: {e}", file=sys.stderr)
            print(f"Query: {sql_query}", file=sys.stderr)
            if params: 
                print(f"Params: {params}", file=sys.stderr)
            return None # Return None on query execution error

# --- Main Execution Block ---
if __name__ == "__main__":
    # Configuration for Jane's Tutoring Business
    DATABASE_FILE = 'TutoringBusiness_GenericOOP.db'
    EXCEL_DATA_FILE = 'tblTutor.xlsx' # Assumed to be in the same directory
    TABLE_NAME = 'tblTutor'
    TABLE_COLUMNS = ['TutorID', 'FirstName', 'LastName', 'Major', 'YearInSchool', 'School', 'HireDate']
    OUTPUT_LOG_FILE = 'tutoring_business_output.txt' # Define the output filename

    # --- Setup Output Redirection ---
    original_stdout = sys.stdout
    original_stderr = sys.stderr
    print(f"--- Script starting. Output will be redirected to '{OUTPUT_LOG_FILE}' ---") # Message to console

    try:
        # Open the output file in write mode, redirect stdout and stderr
        with open(OUTPUT_LOG_FILE, 'w', encoding='utf-8') as outfile:
            sys.stdout = outfile
            sys.stderr = outfile

            # --- Start of Database Operations ---
            print("="*60)
            print(" Tutoring Business DB Management - Generic Methods Execution ")
            print(f" Database file: {DATABASE_FILE}")
            print(f" Excel source:  {EXCEL_DATA_FILE}")
            # Check if Excel file exists before proceeding (optional but good practice)
            if not os.path.exists(EXCEL_DATA_FILE):
                 print(f"\nWARNING: Excel data file not found at specified path: {EXCEL_DATA_FILE}")
                 print("         Import step will likely report a warning or fail.")
            print(f" Output Log:    {OUTPUT_LOG_FILE}")
            print(f" Execution Time: {pd.Timestamp.now()}")
            print("="*60)

            query_results = {} # To store results if needed later

            # Nested try/except for the database operations themselves
            try:
                # Use context manager for automatic connection/disconnection
                with DatabaseManager(db_path=DATABASE_FILE, excel_path=EXCEL_DATA_FILE) as db_manager:

                    # Task 1: Create database file (Handled implicitly by DatabaseManager connection)
                    print("\n--- Task 1: Create Database File (Implicit via connect) ---")
                    # Connection established by `with` statement entering context.
                    # Confirmation should be printed by the connect() method.

                    # Task 2: Create table tblTutor
                    print("\n--- Task 2: Create Table ---")
                    create_table_sql = f"""
                    CREATE TABLE IF NOT EXISTS {TABLE_NAME} (
                        TutorID      INTEGER PRIMARY KEY,
                        FirstName    TEXT NOT NULL,
                        LastName     TEXT NOT NULL,
                        Major        TEXT,
                        YearInSchool TEXT,
                        School       TEXT,
                        HireDate     TEXT
                    );
                    """
                    db_manager.execute_sql(create_table_sql)
                    db_manager.commit() # Commit DDL changes

                    # Task 3: Import data from tblTutor excel file
                    print("\n--- Task 3: Import Data from Excel ---")
                    import_status = db_manager.import_data_from_excel(
                        table_name=TABLE_NAME,
                        expected_columns=TABLE_COLUMNS
                    )
                    # Import method handles its own commit/rollback

                    # Proceed with queries only if import didn't critically fail
                    if not import_status and os.path.exists(EXCEL_DATA_FILE):
                         print("\nCRITICAL: Halting query execution due to data import errors.")
                    else:
                        print("\n--- Proceeding with Data Queries ---")

                        # Task 4: Query tutors hired after April 2017
                        print("\n--- Task 4: Query Tutors Hired After 2017-04-30 ---")
                        sql_hired_after = f"SELECT FirstName, LastName, HireDate FROM {TABLE_NAME} WHERE HireDate > ?"
                        hired_after_threshold = '2017-04-30'
                        results4 = db_manager.execute_select_query(sql_hired_after, (hired_after_threshold,))
                        query_results['hired_after_2017_04'] = results4
                        print("Results:") # Print header even if no results
                        if results4: # Check if list is not None and not empty
                            print(f"{'FirstName':<15} | {'LastName':<15} | {'HireDate'}")
                            print("-" * 47)
                            for row in results4: print(f"{row[0]:<15} | {row[1]:<15} | {row[2]}")
                        elif results4 is None: # Check if query failed
                            print("Query execution failed.")
                        else: # Query succeeded but returned no rows
                            print("No matching records found.")


                        # Task 5: Query distinct types of Major
                        print("\n--- Task 5: Query Distinct Tutor Majors ---")
                        sql_distinct_majors = f"SELECT DISTINCT Major FROM {TABLE_NAME};"
                        results5 = db_manager.execute_select_query(sql_distinct_majors)
                        query_results['distinct_majors'] = results5
                        print("Results (Distinct Majors):") # Print header
                        print("-------------------------")
                        if results5:
                            distinct_majors_list = sorted([str(row[0]) if row[0] is not None else "N/A" for row in results5])
                            for major in distinct_majors_list: print(major)
                        elif results5 is None:
                             print("Query execution failed.")
                        else:
                            print("No distinct majors found.")

                        # Task 6: Query tutors whose YearInSchool is Graduate
                        print("\n--- Task 6: Query Graduate Tutors ---")
                        sql_graduate_tutors = f"SELECT FirstName, LastName FROM {TABLE_NAME} WHERE YearInSchool = ?"
                        graduate_status = 'Graduate'
                        results6 = db_manager.execute_select_query(sql_graduate_tutors, (graduate_status,))
                        query_results['graduate_tutors'] = results6
                        print("Results (Graduate Tutors):") # Print header
                        if results6:
                            print(f"{'FirstName':<15} | {'LastName':<15}")
                            print("-" * 32)
                            for row in results6: print(f"{row[0]:<15} | {row[1]:<15}")
                        elif results6 is None:
                             print("Query execution failed.")
                        else:
                            print("No matching records found.")

                    # Task 7: Download and submit a legible file (Handled by user saving this script and its output)
                    print("\n--- Task 7: Submit Code and Results (User task) ---")
                    print(f"Output log file generated at: {os.path.abspath(OUTPUT_LOG_FILE)}")


            # Catch errors specific to database operations within the 'with' block
            except ConnectionError as ce:
                 print(f"\nDATABASE CONNECTION ERROR: {ce}. Aborting database operations.", file=sys.stderr)
            except sqlite3.Error as sql_e:
                print(f"\nUNHANDLED SQLITE ERROR: {sql_e}. Aborting database operations.", file=sys.stderr)
            except Exception as e:
                 # Catch unexpected errors during database processing
                print(f"\nUNEXPECTED ERROR during database operations: {type(e).__name__} - {e}. Aborting.", file=sys.stderr)
                # import traceback # Uncomment for detailed debugging
                # traceback.print_exc(file=sys.stderr) # Print traceback to the log file

            # This print statement will now go to the file
            print("\n="*2)
            print(" Script execution finished. ")
            print("="*60)
            # --- End of Database Operations ---

    except Exception as e:
        # If error happens *outside* the main db logic (e.g., file open error)
        # Try to print to original stderr
        print(f"FATAL ERROR during script execution or redirection setup: {e}", file=original_stderr)
        # Also attempt to write to the log file if possible, otherwise it's lost
        try:
             with open(OUTPUT_LOG_FILE, 'a', encoding='utf-8') as errfile:
                  print(f"\nFATAL ERROR during script execution or redirection setup: {e}\n", file=errfile)
        except:
             pass # Ignore errors writing the error message itself
    finally:
        # --- Restore Output Streams ---
        sys.stdout = original_stdout
        sys.stderr = original_stderr
        # This final message goes back to the console
        print(f"--- Script finished. Output was redirected to '{OUTPUT_LOG_FILE}' ---")


--- Script starting. Output will be redirected to 'tutoring_business_output.txt' ---
--- Script finished. Output was redirected to 'tutoring_business_output.txt' ---
